Code to run MC_data_collection.py
Clone PyStochasticVolatility first.


In [ ]:
import os 
os.getcwd()
# make sure it is /content

'/content'

In [ ]:
# Clone repository. 
!git clone https://github.com/Dagalon/PyStochasticVolatility.git

Cloning into 'PyStochasticVolatility'...
remote: Enumerating objects: 2172, done.
remote: Counting objects: 100% (249/249), done.
remote: Compressing objects: 100% (174/174), done.
remote: Total 2172 (delta 161), reused 143 (delta 75), pack-reused 1923
Receiving objects: 100% (2172/2172), 3.52 MiB | 9.65 MiB/s, done.
Resolving deltas: 100% (1421/1421), done.


In [ ]:
# Mount Google Drive. 
# your .csv will be saved in your Google Drive root folder.
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
# VERY IMPORTANT.
#!pip install numba-scipy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 18.0 MB/s eta 0:00:00
  Attempting uninstall: scipy
    Found existing installation: scipy 1.10.1
    Uninstalling scipy-1.10.1:
      Successfully uninstalled scipy-1.10.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
arviz 0.15.1 requires scipy>=1.8.0, but you have scipy 1.7.3 which is incompatible.


In [ ]:
# installing dependencies 
# !pip install time 
# !pip install prettytable
# !pip install tabulate
#!pip install sobol_seq

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [38]:
# Upload: MC_data_collection.py to content/PyStochasticVolatility. 
!python /content/PyStochasticVolatility/MC_data_collection.py

# Troubleshooting. Files tab (folder icon, left side of screen) -> PyStochasticVolatility -> MC_data_collection.py. Click on it and it should appear as a pop-up that you can edit. 
# Change num_simulations to 3-5k. 

# For later:
  # Asset prices? 
  # Initializations? 

Options Prices:
 [0.15398407 0.15306352 0.15306352]
Standard Errors:
 [0.00344631 0.00354373 0.00354373]
Deltas:
 [0.02983845 0.12189352 0.12189352]
Gammas:
 [4.92113736e-03 1.84159361e+01 1.84159361e+01]
Asset Prices in First Time Period:
 [100.07221923  99.9694066   99.9694066 ]
Execution Time: 59.40449380874634 seconds
Results saved to CSV files.
